Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [766]:
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed,choice
import numpy as np
from scipy import sparse
from copy import copy

In [767]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [768]:

taboo=[]
step=0;

In [770]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(set_and_size)]),
        )
    )
    return valid, -cost

def fitness(state,sets,set_and_size):
    cost = sum(state)
    valid = np.sum(
        reduce(
            
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(set_and_size)]),
        )
    )
    return valid, -cost


In [771]:
def tweak(state, i, y,set_and_size):
    #TABOO_TECHNIQUE IMPLEMENTED
  
    if(y >=set_and_size):
       y=set_and_size
    new_state = copy(state) 
    while(new_state in taboo):
     new_state = copy(state)
     index = randint(i,y-1)
     new_state[index] = not new_state[index]
    taboo.append(new_state)
    return new_state
def tweak1(state, i, y,set_and_size):
    #TABOO_TECHNIQUE IMPLEMENTED
  
    if(y >=set_and_size):
       y=set_and_size
    new_state = copy(state) 
    stop=0
    while((new_state in taboo) and (stop<set_and_size)):
     stop+=1
     new_state = copy(state)
     index = randint(i,y-1)
     new_state[index] = not new_state[index]
    taboo.append(new_state)
    return new_state

In [772]:
def iterated_local_search(current_state,indexrange,windows_size_value,set_and_size,sets):
    step=0
    improvements=0
    while(fitness(current_state,sets,set_and_size)[0]!=set_and_size)or(improvements<10):
 
    
        new_state = tweak(current_state,indexrange,indexrange+windows_size_value,set_and_size)
        if fitness(new_state,sets,set_and_size) > fitness(current_state,sets,set_and_size):
            current_state = new_state
            improvements=0
            
        else:
            improvements+=1
        step+=1
    return step,fitness(current_state,sets,set_and_size)      

def iterated_local_search2(current_state,indexrange,windows_size_value,set_and_size,sets):
    step=0
    improvements=0
    reset=0

    while(fitness(current_state,sets,set_and_size)[0]!=set_and_size)or(reset<5):
 
    
        new_state = tweak1(current_state,indexrange,indexrange+windows_size_value,set_and_size)
       


        if fitness(new_state,sets,set_and_size) > fitness(current_state,sets,set_and_size):
            current_state = new_state
            improvements=0
            
        else:
            improvements+=1
        
        if((improvements==10)):
            current_state = [choice([False, False, False, False, False, False]) for _ in range(set_and_size)]
            improvements=0
            reset+=1
          
        step+=1
    return step,fitness(current_state,sets,set_and_size)      

In [773]:

num_points_list = [100, 1_000, 5_000]
density_list = [0.3, 0.7]


In [774]:


for num,set_and_size in enumerate(num_points_list):

  # windows_size_vector=[[25,30,50,70,80,90,100],[250,500,800,1000],[2250,2500,3000,4000,5000]]
   windows_size_vector=[[100],[1000],[5000]]
   windows_distance=[25,50,500]

   for density in density_list:
      best_step=set_and_size
      total_step=0 
     
      sets = make_set_covering_problem(set_and_size, set_and_size, density)
      sets=sets.toarray()
      taboo.clear()
   #search_parameter windows_size windows_windows_distance
      indexrange=[i for i in range(0,set_and_size,windows_distance[num])]
      for windows_size in windows_size_vector[num]:
         ##funzina meglio con windowsize=1000 perche parte da una soluzione grande e rimpicciolisce sempre di piu
         for i in indexrange:
            
               current_state = [choice([False, False, False, False, False, False]) for _ in range(set_and_size)]
               best_value=fitness(current_state,sets,set_and_size)
               #print(i,windows_size)    
               
               
               step,value=iterated_local_search2(current_state,i,windows_size,set_and_size,sets)
               total_step+=step
               if step <= best_step:
                        best_step = total_step
                        
               if value >= fitness(best_value,sets,set_and_size):
                 best_value=value
         
      

      print(f"Best_value: {value},FitnessCalltoSolution: {best_step}, TotalFitnessCall: {total_step},num_points: {set_and_size}, Density: {density}")
   
      


Best_value: (100, -11),FitnessCalltoSolution: 100, TotalFitnessCall: 779,num_points: 100, Density: 0.3
Best_value: (100, -4),FitnessCalltoSolution: 328, TotalFitnessCall: 328,num_points: 100, Density: 0.7


Best_value: (1000, -18),FitnessCalltoSolution: 3675, TotalFitnessCall: 3675,num_points: 1000, Density: 0.3
Best_value: (1000, -7),FitnessCalltoSolution: 1856, TotalFitnessCall: 1856,num_points: 1000, Density: 0.7
Best_value: (5000, -24),FitnessCalltoSolution: 2088, TotalFitnessCall: 2088,num_points: 5000, Density: 0.3
Best_value: (5000, -8),FitnessCalltoSolution: 987, TotalFitnessCall: 987,num_points: 5000, Density: 0.7
